In [1]:
import time
import numpy as np
import matplotlib.pyplot as plt
import pandas

import torch
import torch.optim as optim
from torch.nn import CrossEntropyLoss, Conv2d, Sequential, BatchNorm2d
from torch.utils.data import DataLoader

import torchvision
import torchvision.datasets as datasets
from torchvision import transforms

from torchinfo import summary

from ActiveShiftLayer import ASL
from util import test_loss, train_NN

from ray import tune
from ray.tune.search.bayesopt import BayesOptSearch

In [2]:
if torch.cuda.is_available():
    device = 'cuda'
else:
    device = 'cpu'
print(device)

cuda


In [3]:
batch_size = 100

#transform images into normalized tensors
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5,), std=(0.5,))
])

train_dataset = datasets.MNIST(
    "./data/MNIST",
    download=True,
    train=True,
    transform=transform,
)

test_dataset = datasets.MNIST(
    "./data/MNIST",
    download=True,
    train=False,
    transform=transform,
)

train_dataloader = DataLoader(
    dataset=train_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=1,
    pin_memory=True,
)

test_dataloader = DataLoader(
    dataset=test_dataset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=1,
    pin_memory=True,
)

In [5]:
from Models import LeASLNet
input_shape = (batch_size, 1, 28, 28)
criterion = CrossEntropyLoss()

def train_mnist(config):
    test_device = "cpu"
    model = LeASLNet(input_shape, 10, initial_lr=config["lr"], momentum=config["momentum"], weight_decay=config["weight_decay"], device=test_device, expansion_rate=1).to(test_device)
    for i in range(2):
        train_NN(model, criterion, train_dataloader,
        test_dataloader, epochs=2, batches_to_test=100,patience=2,device=test_device, print_test=False, verbose=False)
        acc = test_loss(model, test_dataloader, criterion, test_device)[1]
        tune.report(mean_accuracy=acc)

analysis = tune.run(
    train_mnist,num_samples=15, config={"lr": tune.loguniform(0.001, 0.1), "momentum": tune.uniform(0.8, 1), "weight_decay": tune.uniform(0, 0.1)})

print("Best config: ", analysis.get_best_config(metric="mean_accuracy", mode="max",))

# Get a dataframe for analyzing trial results.
df = analysis.dataframe()

Trial name,status,loc,lr,momentum,weight_decay,acc,iter,total time (s)
train_mnist_44217_00000,TERMINATED,147.142.68.85:68312,0.00446645,0.993003,0.0399922,88.5,2,557.05
train_mnist_44217_00001,TERMINATED,147.142.68.85:68362,0.00252822,0.927413,0.00924895,95.89,2,554.376
train_mnist_44217_00002,TERMINATED,147.142.68.85:68364,0.00452166,0.881696,0.081959,78.7,2,556.69
train_mnist_44217_00003,TERMINATED,147.142.68.85:68368,0.0617364,0.89286,0.0637015,84.1,2,559.309
train_mnist_44217_00004,TERMINATED,147.142.68.85:68394,0.0189816,0.860582,0.0742808,80.58,2,567.034
train_mnist_44217_00005,TERMINATED,147.142.68.85:68420,0.00204454,0.903675,0.086614,78.12,2,567.705
train_mnist_44217_00006,TERMINATED,147.142.68.85:68446,0.0404193,0.881044,0.03029,88.91,2,569.063
train_mnist_44217_00007,TERMINATED,147.142.68.85:68473,0.00115685,0.969497,0.0585754,86.53,2,567.974
train_mnist_44217_00008,TERMINATED,147.142.68.85:68683,0.00100183,0.834387,0.021391,87.59,2,565.338
train_mnist_44217_00009,TERMINATED,147.142.68.85:68816,0.017448,0.936761,0.026766,90.64,2,559.398


2022-08-29 13:46:56,359	WARNING worker.py:1829 -- Warning: The actor ImplicitFunc is very large (52 MiB). Check that its definition is not implicitly capturing a large array or other object in scope. Tip: use ray.put() to put large objects in the Ray object store.
2022-08-29 13:46:56,905	WARNING util.py:220 -- The `start_trial` operation took 0.926 s, which may be a performance bottleneck.
2022-08-29 13:47:01,036	WARNING util.py:220 -- The `start_trial` operation took 0.610 s, which may be a performance bottleneck.
2022-08-29 13:47:01,627	WARNING util.py:220 -- The `start_trial` operation took 0.588 s, which may be a performance bottleneck.
2022-08-29 13:47:02,195	WARNING util.py:220 -- The `start_trial` operation took 0.566 s, which may be a performance bottleneck.
2022-08-29 13:47:08,308	WARNING util.py:220 -- The `start_trial` operation took 0.961 s, which may be a performance bottleneck.
2022-08-29 13:47:16,526	WARNING util.py:220 -- The `start_trial` operation took 0.870 s, which 

Result for train_mnist_44217_00000:
  date: 2022-08-29_13-51-22
  done: false
  experiment_id: c386d6390e1944f5a0fcf79467e8ebc5
  hostname: max-Latitude-5401
  iterations_since_restore: 1
  mean_accuracy: 86.59
  node_ip: 147.142.68.85
  pid: 68312
  time_since_restore: 263.56469988822937
  time_this_iter_s: 263.56469988822937
  time_total_s: 263.56469988822937
  timestamp: 1661773882
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: '44217_00000'
  warmup_time: 0.002355813980102539
  
Result for train_mnist_44217_00001:
  date: 2022-08-29_13-51-29
  done: false
  experiment_id: ea3a79f2a8c04b69be51b1ad36850f88
  hostname: max-Latitude-5401
  iterations_since_restore: 1
  mean_accuracy: 92.61
  node_ip: 147.142.68.85
  pid: 68362
  time_since_restore: 266.13845586776733
  time_this_iter_s: 266.13845586776733
  time_total_s: 266.13845586776733
  timestamp: 1661773889
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: '44217_00001'
  warmup_time: 0.0037386417

2022-08-29 13:59:00,316	INFO tune.py:758 -- Total run time: 731.87 seconds (730.86 seconds for the tuning loop).


Best config:  {'lr': 0.0025282211848313913, 'momentum': 0.9274131119760397, 'weight_decay': 0.009248952703622115}


In [4]:
from Models import LeASLNet
input_shape = (batch_size, 1, 28, 28)
criterion = CrossEntropyLoss()

def train_mnist(config):
    test_device = "cpu"
    model = LeASLNet(input_shape, 10, initial_lr=config["lr"], momentum=config["momentum"], weight_decay=config["weight_decay"], device=test_device, expansion_rate=1).to(test_device)
    for i in range(2):
        train_NN(model, criterion, train_dataloader,
        test_dataloader, epochs=2, batches_to_test=100,patience=2,device=test_device, print_test=False, verbose=False)
        acc = test_loss(model, test_dataloader, criterion, test_device)[1]
        tune.report(mean_accuracy=acc)

analysis = tune.run(
    train_mnist, config={"lr": tune.grid_search([0.001, 0.01, 0.1]), "momentum": tune.grid_search([0.9, 0.99]), "weight_decay": tune.grid_search([0, 0.01, 0.1])})

print("Best config: ", analysis.get_best_config(metric="mean_accuracy", mode="max",))

# Get a dataframe for analyzing trial results.
df = analysis.dataframe()

2022-08-29 14:19:38,106	INFO worker.py:1509 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 
2022-08-29 14:19:39,130	WARNING function_trainable.py:619 -- Function checkpointing is disabled. This may result in unexpected behavior when using checkpointing features or certain schedulers. To enable, set the train function arguments to be `func(config, checkpoint_dir=None)`.


Trial name,status,loc,lr,momentum,weight_decay,acc,iter,total time (s)
train_mnist_dacf6_00000,TERMINATED,147.142.68.85:83875,0.001,0.9,0,91.45,2,536.509
train_mnist_dacf6_00001,TERMINATED,147.142.68.85:83927,0.01,0.9,0,97.93,2,562.865
train_mnist_dacf6_00002,TERMINATED,147.142.68.85:83930,0.1,0.9,0,98.69,2,557.857
train_mnist_dacf6_00003,TERMINATED,147.142.68.85:83956,0.001,0.99,0,97.32,2,558.735
train_mnist_dacf6_00004,TERMINATED,147.142.68.85:83959,0.01,0.99,0,98.02,2,550.848
train_mnist_dacf6_00005,TERMINATED,147.142.68.85:83982,0.1,0.99,0,10.09,2,558.601
train_mnist_dacf6_00006,TERMINATED,147.142.68.85:84160,0.001,0.9,0.01,91.36,2,575.619
train_mnist_dacf6_00007,TERMINATED,147.142.68.85:84163,0.01,0.9,0.01,83.8,2,572.41
train_mnist_dacf6_00008,TERMINATED,147.142.68.85:84349,0.1,0.9,0.01,96.83,2,570.339
train_mnist_dacf6_00009,TERMINATED,147.142.68.85:84404,0.001,0.99,0.01,95.29,2,577.206


2022-08-29 14:19:48,093	WARNING worker.py:1829 -- Warning: The actor ImplicitFunc is very large (52 MiB). Check that its definition is not implicitly capturing a large array or other object in scope. Tip: use ray.put() to put large objects in the Ray object store.
2022-08-29 14:20:32,060	WARNING util.py:220 -- The `start_trial` operation took 1.035 s, which may be a performance bottleneck.


Result for train_mnist_dacf6_00000:
  date: 2022-08-29_14-23-55
  done: false
  experiment_id: 7079a85c0fc04a629399b20373dc3293
  hostname: max-Latitude-5401
  iterations_since_restore: 1
  mean_accuracy: 86.9
  node_ip: 147.142.68.85
  pid: 83875
  time_since_restore: 245.56458640098572
  time_this_iter_s: 245.56458640098572
  time_total_s: 245.56458640098572
  timestamp: 1661775835
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: dacf6_00000
  warmup_time: 0.0023016929626464844
  
Result for train_mnist_dacf6_00002:
  date: 2022-08-29_14-24-17
  done: false
  experiment_id: 155fc01774114092ba5f488a31e405a0
  hostname: max-Latitude-5401
  iterations_since_restore: 1
  mean_accuracy: 97.72
  node_ip: 147.142.68.85
  pid: 83930
  time_since_restore: 262.0902826786041
  time_this_iter_s: 262.0902826786041
  time_total_s: 262.0902826786041
  timestamp: 1661775857
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: dacf6_00002
  warmup_time: 0.00828075408935546

2022-08-29 14:34:38,071	INFO tune.py:758 -- Total run time: 898.94 seconds (897.58 seconds for the tuning loop).


Best config:  {'lr': 0.1, 'momentum': 0.9, 'weight_decay': 0}


In [5]:
from Models import LeNet
input_shape = (batch_size, 1, 28, 28)
criterion = CrossEntropyLoss()

def train_mnist(config):
    test_device = "cpu"
    model = LeNet(input_shape, 10, initial_lr=config["lr"], momentum=config["momentum"], weight_decay=config["weight_decay"]).to(test_device)
    for i in range(2):
        train_NN(model, criterion, train_dataloader,
        test_dataloader, epochs=2, batches_to_test=100,patience=2,device=test_device, print_test=False, verbose=False)
        acc = test_loss(model, test_dataloader, criterion, test_device)[1]
        tune.report(mean_accuracy=acc)

analysis = tune.run(
    train_mnist, config={"lr": tune.grid_search([0.001, 0.01, 0.1]), "momentum": tune.grid_search([0.9, 0.99]), "weight_decay": tune.grid_search([0, 0.01, 0.1])})

print("Best config: ", analysis.get_best_config(metric="mean_accuracy", mode="max",))

# Get a dataframe for analyzing trial results.
df = analysis.dataframe()

Trial name,status,loc,lr,momentum,weight_decay,acc,iter,total time (s)
train_mnist_d6d05_00000,TERMINATED,147.142.68.85:96840,0.001,0.9,0,95.77,2,430.805
train_mnist_d6d05_00001,TERMINATED,147.142.68.85:96891,0.01,0.9,0,94.67,2,444.224
train_mnist_d6d05_00002,TERMINATED,147.142.68.85:96898,0.1,0.9,0,99.07,2,444.087
train_mnist_d6d05_00003,TERMINATED,147.142.68.85:96920,0.001,0.99,0,98.35,2,450.634
train_mnist_d6d05_00004,TERMINATED,147.142.68.85:96922,0.01,0.99,0,98.67,2,438.771
train_mnist_d6d05_00005,TERMINATED,147.142.68.85:96962,0.1,0.99,0,30.74,2,444.898
train_mnist_d6d05_00006,TERMINATED,147.142.68.85:97124,0.001,0.9,0.01,96.17,2,461.305
train_mnist_d6d05_00007,TERMINATED,147.142.68.85:97127,0.01,0.9,0.01,93.32,2,460.013
train_mnist_d6d05_00008,TERMINATED,147.142.68.85:97310,0.1,0.9,0.01,97.27,2,455.669
train_mnist_d6d05_00009,TERMINATED,147.142.68.85:97393,0.001,0.99,0.01,95.66,2,457.808


2022-08-29 14:55:29,402	WARNING worker.py:1829 -- Warning: The actor ImplicitFunc is very large (52 MiB). Check that its definition is not implicitly capturing a large array or other object in scope. Tip: use ray.put() to put large objects in the Ray object store.
2022-08-29 14:55:29,523	WARNING util.py:220 -- The `start_trial` operation took 0.875 s, which may be a performance bottleneck.
2022-08-29 14:55:39,059	WARNING util.py:220 -- The `start_trial` operation took 0.712 s, which may be a performance bottleneck.
2022-08-29 14:55:39,790	WARNING util.py:220 -- The `start_trial` operation took 0.728 s, which may be a performance bottleneck.
2022-08-29 14:55:45,867	WARNING util.py:220 -- The `start_trial` operation took 0.784 s, which may be a performance bottleneck.
2022-08-29 14:55:52,663	WARNING util.py:220 -- The `start_trial` operation took 0.834 s, which may be a performance bottleneck.
2022-08-29 14:55:59,831	WARNING util.py:220 -- The `start_trial` operation took 0.896 s, which 

Result for train_mnist_d6d05_00000:
  date: 2022-08-29_14-58-48
  done: false
  experiment_id: 6eb2d6ddb1ca433a98476d01ce30bb94
  hostname: max-Latitude-5401
  iterations_since_restore: 1
  mean_accuracy: 91.01
  node_ip: 147.142.68.85
  pid: 96840
  time_since_restore: 196.7659192085266
  time_this_iter_s: 196.7659192085266
  time_total_s: 196.7659192085266
  timestamp: 1661777928
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: d6d05_00000
  warmup_time: 0.0029163360595703125
  
Result for train_mnist_d6d05_00001:
  date: 2022-08-29_14-59-00
  done: false
  experiment_id: 09dead0ff7394abfba708c66d4c6fb7e
  hostname: max-Latitude-5401
  iterations_since_restore: 1
  mean_accuracy: 84.55
  node_ip: 147.142.68.85
  pid: 96891
  time_since_restore: 205.38097524642944
  time_this_iter_s: 205.38097524642944
  time_total_s: 205.38097524642944
  timestamp: 1661777940
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: d6d05_00001
  warmup_time: 0.0028080940246582

2022-08-29 15:07:06,676	INFO tune.py:758 -- Total run time: 706.66 seconds (705.67 seconds for the tuning loop).


Best config:  {'lr': 0.1, 'momentum': 0.9, 'weight_decay': 0}


In [4]:
from Models import LeNet
input_shape = (batch_size, 1, 28, 28)
criterion = CrossEntropyLoss()

def train_mnist(config):
    print(config)
    test_device = "cpu"
    model = LeNet(input_shape, 10, initial_lr=config["lr"], momentum=config["momentum"], weight_decay=0).to(test_device)
    for i in range(2):
        train_NN(model, criterion, train_dataloader,
        test_dataloader, epochs=2, batches_to_test=100,patience=2,device=test_device, print_test=False, verbose=False)
        acc = test_loss(model, test_dataloader, criterion, test_device)[1]
    tune.report(mean_accuracy=acc)

In [5]:
config = {"lr": tune.uniform(0.001, 0.1), "momentum": tune.uniform(0.1, 1)}

bayesopt = BayesOptSearch(metric="mean_accuracy", mode="max", verbose=1, random_state=36)

tuner = tune.Tuner(train_mnist, tune_config=tune.TuneConfig(search_alg=bayesopt, num_samples=20), param_space=config)

analysis = tuner.fit()

2022-08-30 13:12:48,199	INFO worker.py:1509 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8268 
2022-08-30 13:12:49,626	WARNING function_trainable.py:619 -- Function checkpointing is disabled. This may result in unexpected behavior when using checkpointing features or certain schedulers. To enable, set the train function arguments to be `func(config, checkpoint_dir=None)`.
2022-08-30 13:12:50,317	WARNING bayesopt_search.py:423 -- BayesOpt does not support specific sampling methods. The Uniform sampler will be dropped.
2022-08-30 13:12:50,318	WARNING bayesopt_search.py:423 -- BayesOpt does not support specific sampling methods. The Uniform sampler will be dropped.


Trial name,status,loc,lr,momentum
train_mnist_afb6cf86,RUNNING,129.206.61.139:89545,0.0731222,0.641453
train_mnist_b1f6e3da,RUNNING,129.206.61.139:89593,0.0952659,0.228856
train_mnist_b2afae10,RUNNING,129.206.61.139:89616,0.0809286,0.426893
train_mnist_b357a00c,RUNNING,129.206.61.139:89653,0.0194691,0.333236


2022-08-30 13:12:52,384	WARNING worker.py:1829 -- Warning: The actor ImplicitFunc is very large (52 MiB). Check that its definition is not implicitly capturing a large array or other object in scope. Tip: use ray.put() to put large objects in the Ray object store.


(train_mnist pid=89545) {'lr': 0.07312221196797684, 'momentum': 0.641452791133192}


2022-08-30 13:12:59,599	WARNING tune.py:686 -- Stop signal received (e.g. via SIGINT/Ctrl+C), ending Ray Tune run. This will try to checkpoint the experiment state one last time. Press CTRL+C (or send SIGINT/SIGKILL/SIGTERM) to skip. 


(train_mnist pid=89593) {'lr': 0.09526588607319536, 'momentum': 0.22885610294892744}
(train_mnist pid=89616) {'lr': 0.08092855385714631, 'momentum': 0.42689313686892605}
(train_mnist pid=89653) {'lr': 0.019469124416642274, 'momentum': 0.33323611445915247}


In [18]:
print(analysis.get_best_result(metric="mean_accuracy", mode="max"))
print(analysis.get_best_result(metric="mean_accuracy", mode="max").config)
analysis.get_dataframe()

Result(metrics={'mean_accuracy': 99.02, 'done': True, 'trial_id': '6a1cdb10', 'experiment_tag': '20_lr=0.0473,momentum=0.9375'}, error=None, log_dir=PosixPath('/home/max/ray_results/train_mnist_2022-08-30_12-18-52/train_mnist_6a1cdb10_20_lr=0.0473,momentum=0.9375_2022-08-30_12-27-58'))
{'lr': 0.04729373516435155, 'momentum': 0.9374638569924613}


,mean_accuracy,time_this_iter_s,done,timesteps_total,episodes_total,training_iteration,trial_id,experiment_id,date,timestamp,...,pid,hostname,node_ip,time_since_restore,timesteps_since_restore,iterations_since_restore,warmup_time,config/lr,config/momentum,logdir
0,98.90,444.894855,False,NaN,NaN,1,25f94c3a,373471c29ae84f63b956e49ed4c9aa76,2022-08-30_12-26-21,1661855181,...,65266,max-Latitude-5401,147.142.66.195,444.894855,0,1,0.002728,0.073122,0.641453,/home/max/ray_results/train_mnist_2022-08-30_1...
1,98.70,450.712692,False,NaN,NaN,1,28212870,a424642e8dfc4ab5a70ce9f93978c289,2022-08-30_12-26-31,1661855191,...,65340,max-Latitude-5401,147.142.66.195,450.712692,0,1,0.003052,0.095266,0.228856,/home/max/ray_results/train_mnist_2022-08-30_1...
2,96.73,459.397322,False,NaN,NaN,1,28b2483c,fdfea3717dfd4b0695070db430d91ec9,2022-08-30_12-26-41,1661855201,...,65343,max-Latitude-5401,147.142.66.195,459.397322,0,1,0.005404,0.080929,0.426893,/home/max/ray_results/train_mnist_2022-08-30_1...
3,97.01,458.812992,False,NaN,NaN,1,294cdc9e,9d952fee6de84c81bd1c590655250a88,2022-08-30_12-26-41,1661855201,...,65370,max-Latitude-5401,147.142.66.195,458.812992,0,1,0.003459,0.019469,0.333236,/home/max/ray_results/train_mnist_2022-08-30_1...
4,98.15,461.432683,False,NaN,NaN,1,2bc087b4,1d686e12b8554adb935b2ab97cd8bd29,2022-08-30_12-26-50,1661855210,...,65497,max-Latitude-5401,147.142.66.195,461.432683,0,1,0.004276,0.063346,0.129305,/home/max/ray_results/train_mnist_2022-08-30_1...
5,99.00,459.758437,False,NaN,NaN,1,2cb8037c,39ac369514044e9ea32dda6a1e47bb3e,2022-08-30_12-26-50,1661855210,...,65516,max-Latitude-5401,147.142.66.195,459.758437,0,1,0.007722,0.094776,0.913886,/home/max/ray_results/train_mnist_2022-08-30_1...
6,98.71,457.448251,False,NaN,NaN,1,30469b02,ce2650e570e74d0f8ded8b97fa8382c5,2022-08-30_12-26-57,1661855217,...,65656,max-Latitude-5401,147.142.66.195,457.448251,0,1,0.011883,0.055463,0.277307,/home/max/ray_results/train_mnist_2022-08-30_1...
7,98.08,448.175605,False,NaN,NaN,1,35db6958,c0b398878bde432b83b97d852bc6c850,2022-08-30_12-26-58,1661855218,...,65827,max-Latitude-5401,147.142.66.195,448.175605,0,1,0.016170,0.023971,0.102129,/home/max/ray_results/train_mnist_2022-08-30_1...
8,97.82,440.985883,False,NaN,NaN,1,3c0a95d8,1f9aa92842d54ddbae54e826ab0eb616,2022-08-30_12-27-02,1661855222,...,65918,max-Latitude-5401,147.142.66.195,440.985883,0,1,0.021348,0.023379,0.437809,/home/max/ray_results/train_mnist_2022-08-30_1...
9,68.10,429.602866,False,NaN,NaN,1,42963ff6,6ec618686c374872bc355aa0b21c905d,2022-08-30_12-27-03,1661855223,...,66072,max-Latitude-5401,147.142.66.195,429.602866,0,1,0.025281,0.045822,0.321471,/home/max/ray_results/train_mnist_2022-08-30_1...
